# Health Care Analysis - MENA Countries

## Overview

This project analyzes healthcare spending efficiency across MENA countries, focusing on how effectively per-capita health expenditure translates into health outcomes.
The analysis adopts a consulting-style approach to support decision-making, benchmarking, and prioritization.

# Import Required Libraries

Import pandas for data manipulation, numpy for numerical operations, and re for regular expressions.

In [1]:
import pandas as pd
import numpy as np
import re

# Load Data

Load the health care dataset from the CSV file.

In [2]:
# Define the path to the dataset
path = "P_Data_Extract_From_Health_Nutrition_and_Population_Statistics\\Health Care dataset.csv"

# Load the raw data
df_raw = pd.read_csv(path)

# Define Indicators

Map the series codes to readable indicator names.

In [3]:
# Mapping of series codes to indicator names
indicators = {
    "SH.XPD.CHEX.GD.ZS": "Health_Exp_GDP",           # % of GDP
    "SH.XPD.CHEX.PP.CD": "Health_Exp_Per_Capita_PPP", # per capita PPP (international $)
    "SH.MED.PHYS.ZS": "Physicians_per_1000",
    "SH.MED.BEDS.ZS": "Beds_per_1000",
    "SP.DYN.LE00.IN": "Life_Expectancy",
    "SP.DYN.IMRT.IN": "Infant_Mortality",
}

# Data Cleaning and Transformation

Process the data: identify year columns, filter relevant indicators, transform to long format, clean values, and pivot to clean dataset.

In [4]:
# Identify year columns using regex
year_cols = [c for c in df_raw.columns if re.match(r"^\d{4}\s+\[YR\d{4}\]$", str(c))]

In [5]:
# Filter the dataframe to keep only relevant indicators and columns
df = df_raw.loc[df_raw["Series Code"].isin(indicators.keys()),
                ["Country Name", "Country Code", "Series Code"] + year_cols].copy()

In [6]:
# Transform from wide to long format
df_long = df.melt(
    id_vars=["Country Name", "Country Code", "Series Code"],
    value_vars=year_cols,
    var_name="Year",
    value_name="Value",
)

In [7]:
# Extract the year as integer from the column name
df_long["Year"] = df_long["Year"].str.extract(r"^(\d{4})").astype(int)

In [8]:
# Convert values to numeric, treating ".." as NaN
df_long["Value"] = pd.to_numeric(df_long["Value"].replace("..", np.nan), errors="coerce")

In [9]:
# Map series codes to indicator names
df_long["Indicator"] = df_long["Series Code"].map(indicators)

In [10]:
# Pivot to create a clean dataset with indicators as columns
df_clean = (
    df_long.pivot_table(
        index=["Country Name", "Country Code", "Year"],
        columns="Indicator",
        values="Value",
        aggfunc="first",
    )
    .reset_index()
    .rename(columns={"Country Name": "Country", "Country Code": "Country_Code"})
    .sort_values(["Country", "Year"])
)

# Create Latest Snapshots

Create datasets for the latest available data per country.

In [11]:
# Dataset for latest year per country based on Life Expectancy availability
latest_by_country = (
    df_clean.dropna(subset=["Life_Expectancy"])
    .sort_values(["Country", "Year"])
    .groupby("Country", as_index=False)
    .tail(1)
    .reset_index(drop=True)
)

In [18]:
# Alternative snapshot: fill forward/backward and take last per country
latest_snapshot = (
    df_clean
    .sort_values(["Country", "Year"])
    .groupby("Country")
    .apply(lambda x: x.ffill().bfill().iloc[-1], include_groups=True)
    .reset_index(drop=True)
)

C:\Users\asemh\AppData\Local\Temp\ipykernel_9480\1393708873.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.ffill().bfill().iloc[-1], include_groups=True)


# Save Data

Save the cleaned datasets to CSV files for further analysis.

In [19]:
# Save cleaned datasets to CSV
df_clean.to_csv("mena_health_clean_long.csv", index=False)
latest_by_country.to_csv("mena_health_latest_by_country.csv", index=False)
latest_snapshot.to_csv("mena_health_latest_snapshot.csv", index=False)

# Print shapes and display head of df_clean
print("df_clean shape:", df_clean.shape)
print("latest_by_country shape:", latest_by_country.shape)
print("latest_snapshot shape:", latest_snapshot.shape)

df_clean shape: (140, 9)
latest_by_country shape: (10, 9)
latest_snapshot shape: (10, 9)


# Analysis

Perform efficiency analysis on the latest snapshot data.

In [ ]:
# Select relevant columns for efficiency analysis
efficiency_df = latest_snapshot[[
    "Country",
    "Health_Exp_Per_Capita_PPP",
    "Life_Expectancy",
    "Infant_Mortality",
    "Physicians_per_1000",
    "Beds_per_1000"
]].copy()

efficiency_df

In [ ]:
# Calculate MENA region averages for key indicators
mena_avg = efficiency_df.mean(numeric_only=True)
mena_avg

In [ ]:
# Define a function to categorize efficiency based on spending and outcomes
def efficiency_category(row):
    spend = row["Health_Exp_Per_Capita_PPP"]
    outcome = row["Life_Expectancy"]
    
    if spend >= mena_avg["Health_Exp_Per_Capita_PPP"] and outcome >= mena_avg["Life_Expectancy"]:
        return "High Efficiency"
    elif spend >= mena_avg["Health_Exp_Per_Capita_PPP"] and outcome < mena_avg["Life_Expectancy"]:
        return "Efficiency Gap"
    elif spend < mena_avg["Health_Exp_Per_Capita_PPP"] and outcome >= mena_avg["Life_Expectancy"]:
        return "Best Practice"
    else:
        return "Under-resourced"

# Apply the categorization
efficiency_df["Efficiency_Category"] = efficiency_df.apply(efficiency_category, axis=1)
efficiency_df

In [ ]:
# Sort by Infant Mortality to see countries with lowest rates
efficiency_df.sort_values("Infant_Mortality")

##  Insights
### Insight 1 Variation in Efficiency

Healthcare spending efficiency varies significantly across MENA countries, indicating that higher expenditure does not consistently translate into better outcomes.

### Insight 2 – Best Practices

Several countries achieve above-average life expectancy with below-average per-capita health expenditure, highlighting scalable efficiency best practices.

### Insight 3 – Efficiency Gaps

A subset of high-spending countries underperforms on key health outcomes, suggesting opportunities for targeted resource reallocation and governance improvements.